## 파일 설명
| 파일명 | 파일 용도 | 관련 절 | 페이지 |
|:--   |:--      |:--    |:--      |
| train_neuralnet.py | 4장의 train_neuralnet.py와 같습니다. 단, 수치 미분 대신 오차역전파법으로 기울기를 구합니다. | 5.7.4 오차역전파법을 사용한 학습 구현하기 | 186 |
| two_layer_net.py | 오차역전파법을 적용한 2층 신경망 클래스 | 5.7.2 오차역전파법을 적용한 신경망 구현하기 | 181 |

## 5장 오차역전파법

## 목차
```
5.6 Affine/Softmax 계층 구현하기 
__5.6.1 Affine 계층 
__5.6.2 배치용 Affine 계층 
__5.6.3 Softmax-with-Loss 계층 
```


In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

### 5.6.1 Affine 계층
![그림 5-23](../deep_learning_images/fig 5-23.png)

In [ ]:
X = np.random.rand(2) # 입력
W = np.random.rand(2,3) # 가중치
B = np.random.rand(3) # 편향

print("X의 차원 : ", X.shape)
print("W의 차원 : ", W.shape)
print("B의 차원 : ", B.shape)

![그림 5-24](../deep_learning_images/fig 5-24.png)

In [ ]:
Y = np.dot(X, W) + B
print("X : ", X)
print("W : ", W)
print("B : ", B)
print("XW+B = Y : ", Y)

In [ ]:
import tensorflow as tf
X = tf.placeholder(tf.float64, shape=(1,2))
W = tf.placeholder(tf.float64, shape=(2,3))
B = tf.constant(np.random.rand(3))
Y = tf.add(tf.matmul(X,W), B)

with tf.Session() as sess:
    result = sess.run(Y, feed_dict={X: np.random.rand(1,2), W: np.random.rand(2,3)})
    print(result)

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


### 5.6.3 Softmax-with-Loss 계층
![그림 5-30](../deep_learning_images/fig 5-30.png)

In [ ]:
def softmax(a):  ## from chapter 3 page 91-93
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a    
    return y

def cross_entropy_error(y, t):  ## from chapter 4 page 113, 114
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

### Tensorflow version

In [ ]:
x = tf.placeholder(tf.float64, shape=(1,2), name='input')
w = tf.placeholder(tf.float64, shape=(2,3), name='weight')
b = tf.constant(np.random.rand(1,3), name='bias')
t = tf.constant(np.random.rand(1,3), name="label")
y = tf.add(tf.matmul(x,w), b)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    result = sess.run(y, feed_dict={x: np.random.rand(1,2), w: np.random.rand(2,3)})    
    loss = tf.losses.softmax_cross_entropy(y, t)
